<div align="center">

## Proyecto Modulo 1.- El **TITANIC** *Procesamiento de datos*

<div>

##### Importamos librerias necesarias

In [38]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn.impute import KNNImputer
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
from IPython.display import Image, display, HTML
from sklearn.preprocessing import LabelEncoder
import warnings 
warnings.filterwarnings('ignore')

In [ ]:
# leemos csv
pd.read_csv('')

##### Creamos 2 df para tratar los datos nulos bajo dos posibilidades diferentes y analizaremos cual es la mas efectiva para trabajar sobre ellos.

In [40]:
df_media = pd.read_csv('')
df_Knn = pd.read_csv('')

##### Empezaremos a trabajar sobre 'df_media'

In [ ]:
df_media.head()

In [ ]:
df_media.columns

In [43]:
# eliminamos posibles datos duplicados
df_media.drop_duplicates(inplace=True)

In [ ]:
# visualizamos las columnas con datos nulos
df_media.isnull().sum() / df_media.shape[0]*100

##### Preprocesamiento sobre la columna 'Age' contiene un 19.8% de valores nulos, en este caso vamos a completar con la media del resto del conjunto de datos.

In [ ]:
mean_age = df_media['Age'].mean()
mean_age
df_media['Age'].fillna(mean_age, inplace=True)
print(df_media['Age'].isnull().sum)

##### Visualizamos el gráfico de distribución de la edad después de rellenar los valores nulos con la media.

In [ ]:
plt.figure(figsize=(10, 6))

# Gráfico de distribución de la edad después de rellenar valores nulos
sns.histplot(df_media['Age'], kde=True, color='blue', label='Edad con valores nulos rellenados')

# Añadir la línea de la media
plt.axvline(mean_age, color='red', linestyle='--', label=f'Media: {mean_age:.2f} años')

# Añadir leyenda y título
plt.legend()
plt.title('Distribución de Edad en el Dataset del Titanic')
plt.xlabel('Edad')
plt.ylabel('Frecuencia')

## Relleno de Valores Nulos

### Método Aplicado
Rellenamos los valores nulos de la columna `Age` con la media de 29.70 años.

### Resultado
Generamos una visualización usando `sns.histplot` para mostrar la distribución de las edades después de esta imputación.

### Observaciones
Rellenar los datos nulos con la media resultó en una sobresaturación de la edad media (29.70 años), lo que no es representativo de una población diversa. Este método es sencillo pero no captura la variabilidad de los datos reales.

### Propuesta de Mejora
Para obtener estimaciones más precisas de las edades faltantes, proponemos el desarrollo de un modelo K-Nearest Neighbors (KNN). Este enfoque utilizará las características disponibles en el conjunto de datos para predecir la edad de manera más cercana a los valores reales, manteniendo la integridad estadística y mejorando la precisión del análisis.

## Próximos Pasos
1. **Desarrollo del Modelo KNN:** Entrenar y validar el modelo utilizando datos completos.
2. **Evaluación de Resultados:** Comparar la precisión del modelo KNN frente al método de imputación por media.
3. **Implementación:** Aplicar el modelo KNN al conjunto de datos y analizar los resultados.
   

In [ ]:
df_Knn.head()

In [48]:
# eliminamos duplicados 
df_Knn.drop_duplicates(inplace=True)

In [ ]:
df_Knn.isnull().sum() 

In [50]:
# Vamos a normalizar Fare para asegurarnos que no tenga un impacto desproporcionado en la distancia 
scaler = StandardScaler()
df_Knn['Fare'] = scaler.fit_transform(df_Knn[['Fare']])

In [51]:
# estas son las columnas que elegimos como predictores de nuestro modelo.

columnas=['Survived', 'Pclass', 'SibSp', 'Parch', 'Fare', 'Embarked',"Age"] #la edad tiene que tenerse en cuenta al predecir, los apellidos es una variable muy random y ruidosa para introducirse

# Aplicamos OneHotEncoder para codificar nuestras variables categóricas 
encoder = OneHotEncoder(drop='first', sparse_output=False)
df_encoded = pd.DataFrame(encoder.fit_transform(df_Knn[columnas]))
df_encoded.columns = encoder.get_feature_names_out(columnas)

In [52]:
# Creamos dos dataframes basados en la presencia de Age
df_encoded['Age'] = df_Knn['Age']
df_with_age = df_encoded.dropna(subset=['Age'])
df_without_age = df_encoded[df_encoded['Age'].isna()].drop(columns='Age')

In [53]:
# Dividimos el conjunto de datos en un conjunto de entrenamiento y un conjunto de prueba
X_train, X_test, y_train, y_test = train_test_split(df_with_age.drop(columns='Age'), df_with_age['Age'], test_size=0.2, random_state=357)

In [ ]:
# Lista para almacenar los valores de MSE (Error Cuadrático Medio)
mse = []

# Rango de k para probar
k_range = range(1, 7)

for k in k_range:
    knn = KNeighborsRegressor(n_neighbors=k)
    
    # Realizamos la validación cruzada para obtener una medida más robusta del error
    """  estamos tomando el negativo de los valores devueltos para obtener el MSE real. 
         Luego, añadimos la media de estos valores a nuestra lista de MSE para cada valor de k (mse.append(scores.mean())).
    """
    scores = -cross_val_score(knn, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    mse.append(scores.mean())
    print(f'scores = {scores}')
    
 
# Graficamos los valores 
plt.plot(k_range, mse)
plt.xlabel('k')
plt.ylabel('MSE')
plt.title('Regla del codo para determinar el valor óptimo de k')
plt.show()


In [ ]:
# Determinar el mejor valor de k
best_k = k_range[mse.index(min(mse))]
print(f'Mejor número K: {best_k}')

In [56]:
# Creamos el modelo KNN con el mejor valor de k
knn = KNeighborsRegressor(n_neighbors=best_k)

# Ajustamos el modelo a los datos sin valores nulos
knn.fit(X_train, y_train)

# Imputamos los valores faltantes en la columna 'Age'
imputed_ages = knn.predict(df_without_age)
df_Knn.loc[df_Knn['Age'].isna(), 'Age'] = imputed_ages

In [ ]:
plt.figure(figsize=(10, 6))

# Gráfico de distribución de la edad después de rellenar valores nulos
sns.histplot(df_Knn['Age'], kde=True, color='blue', label='Edad con valores nulos rellenados')

# Añadir la línea de la media
plt.axvline(mean_age, color='red', linestyle='--', label=f'Media: {mean_age:.2f} años')

# Añadir leyenda y títulos
plt.legend()
plt.title('Distribución de Edad en el Dataset del Titanic')
plt.xlabel('Edad')
plt.ylabel('Frecuencia')

#### **Conclusión**

##### A través de la implementación del modelo KNN, esperamos mejorar significativamente la precisión de nuestras imputaciones de edad, reflejando mejor la diversidad demográfica del Titanic. Este cambio no solo mejora la calidad del análisis, sino que también proporciona una base más sólida para futuros estudios y modelos predictivos.

In [ ]:


# Cargar las imágenes
img1 = mpimg.imread('')
img2 = mpimg.imread('')

# Crear una figura y ejes
fig, axs = plt.subplots(1, 2, figsize=(12, 6))

# Mostrar la primera imagen
axs[0].imshow(img1)
axs[0].axis('off')
axs[0].set_title('KNN')

# Mostrar la segunda imagen
axs[1].imshow(img2)
axs[1].axis('off')
axs[1].set_title('Media')

# Mostrar el gráfico
plt.show()


##### Reparación de valores nulos de la columna 'Embarked' a través de la implementación de modelo KNN.

In [ ]:

# Seleccionar las columnas relevantes
columns = ['Embarked', 'Pclass', 'Age', 'Fare']
df_selected = df_Knn[columns]

# Convertir los valores categóricos a numéricos
df_selected['Embarked'] = df_selected['Embarked'].map({'C': 0, 'Q': 1, 'S': 2})

# Crear el imputador KNN
imputer = KNNImputer(n_neighbors=5)

# Imputar los valores nulos
df_selected = pd.DataFrame(imputer.fit_transform(df_selected), columns=columns)

# Convertir los valores numéricos de 'Embarked' de vuelta a categóricos
df_selected['Embarked'] = df_selected['Embarked'].round().map({0: 'C', 1: 'Q', 2: 'S'})

# Reemplazar la columna original en el DataFrame
df_Knn['Embarked'] = df_selected['Embarked']

# Verificar los cambios
print(df_Knn['Embarked'].isnull().sum())


In [ ]:
plt.figure(figsize=(10, 6))

# Gráfico de distribución de la edad después de rellenar valores nulos
sns.histplot(df_Knn['Embarked'], kde=True, color='blue', label='Embarked valores nulos rellenados')



# Añadir leyenda y títulos
plt.legend()
plt.title('Distribución de Embarked en el Dataset del Titanic')
plt.xlabel('Embarked')
plt.ylabel('Frecuencia')

In [ ]:
df_Knn.isnull().sum() / df_media.shape[0]*100

##### Desechamos la columna Cabin ya que tiene un 70% de datos nulos y carece de sentido repararla

In [62]:
df_Knn.drop(columns=['Cabin'], inplace=True)

In [ ]:
df_Knn.columns

##### Desechamos tambien la columna 'PassengerId'

In [64]:
df_Knn.drop(columns=['PassengerId'], inplace=True)

In [ ]:
df_Knn.columns

##### Vamos a crear una columna nueva con el nombre 'LastName' en la que sacaremos el apellido de los viajeros.

In [66]:
df_Knn['LastName'] = df_Knn['Name'].apply(lambda x: x.split(',')[0])

In [ ]:
df_Knn.head()

##### Normalizamos la columna 'Ticket'.

In [ ]:
label_encoder = LabelEncoder()

# Codificar la columna 'Ticket'
df_Knn['Ticket'] = label_encoder.fit_transform(df_Knn['Ticket'])
df_Knn.head()

##### Cambiamos la columna 'Survived' a booleano

In [ ]:
# Cambiar la columna 'Survived' a booleana
df_Knn['Survived'] = df_Knn['Survived'].astype(bool)
df_Knn.head()

##### Reparamos la columna AGE para verla solo con dos cifras

In [ ]:
# Mantener solo las dos primeras cifras de la columna 'Age'
def truncate_age(age):
    if pd.notnull(age):
        age_str = str(age).split('.')[0]  # Quitar la parte decimal si existe
        return int(age_str[:2])
    return age

df_Knn['Age'] = df_Knn['Age'].apply(truncate_age)
df_Knn.head()


##### Guardar el DataFrame df_Knn como un archivo CSV con el que trabajaremos en el EDA.

In [ ]:
df_Knn.to_csv('', index=False)


## Procesamiento de Datos para el Proyecto Titanic 🚢

### Relleno de Valores Nulos
**Método Aplicado**: Rellenamos los valores nulos de la columna `Age` con la media de 29.70 años.

**Resultado**: Generamos una visualización utilizando `sns.histplot` para mostrar la distribución de las edades después de esta imputación.

**Observaciones**: Rellenar los datos nulos con la media resultó en una sobresaturación de la edad media (29.70 años), lo que no es representativo de una población diversa. Este método es sencillo pero no captura la variabilidad de los datos reales.

### Propuesta de Mejora
Para obtener estimaciones más precisas de las edades faltantes, proponemos el desarrollo de un modelo K-Nearest Neighbors (KNN). Este enfoque utilizará las características disponibles en el conjunto de datos para predecir la edad de manera más cercana a los valores reales, manteniendo la integridad estadística y mejorando la precisión del análisis.

### Implementación del Modelo KNN
1. **Desarrollo del Modelo KNN**: Entrenar y validar el modelo utilizando datos completos.
2. **Evaluación de Resultados**: Comparar la precisión del modelo KNN frente al método de imputación por media.
3. **Implementación**: Aplicar el modelo KNN al conjunto de datos y analizar los resultados.

### Otras reparaciones:
1. Reducir a visualizar con solo  2 cifras
2. Desechamos la columna 'Cabin' ya que contiene 70% de datos nulos
3. Desechamos la columna 'PassengerId' ya que no nos aporta ningun valor al EDA.
4. Creamos columna 'Last Name' para poder relacionar miembros de grupos familiares.
5. Normailizamos la columna 'Ticket' ya que no hemos encotrado nada concluyente acorde a los prefijos de los tickets.

---

